# **Liens utiles**

## Liens officiels IPN database :


*   Site web officiel : https://gibranbenitez.github.io/IPN_Hand/
*   Dossier Google Drive général : https://drive.google.com/drive/folders/1aL645mUzzAvoTMwJKrbtQiNiDVJZ2EsA
*   Dossier Google Drive vidéos : https://drive.google.com/drive/folders/1O4Fn_jbAEcKIksXHmQIMcHTyaDw4wt9x
*   Fichier annotations pour l'entrainement : https://drive.google.com/file/d/1Y6yIcckNonsRZmyD833oCLF-ndu3rvT7
*   Fichier annotations pour le test : https://drive.google.com/file/d/13fXL1r62yuVDn6P-QR7GYM4L7d5IdrZa


## Liens des résultats des travaux sur IPN :



*   Dossier Google Drive général : https://drive.google.com/drive/folders/1iNQrCp5XM-SmbThc52NASfIacgMJkxml
*   Dossier Google Drive landmarks : https://drive.google.com/drive/folders/1DP1tUXELlu-RrKeVJsHx7KVdkl7x3WpI
*   Dossier Google Drive vidéos annotées : https://drive.google.com/drive/folders/1p682ZGJV6MpPsZl8yRFHwfQjcKkbvbkB





**Téléchargement des fichiers**

In [ ]:
!rm -rf sample_data
!pip install -q --upgrade --no-cache-dir gdown

# Landmarks
!gdown --folder https://drive.google.com/drive/folders/1p682ZGJV6MpPsZl8yRFHwfQjcKkbvbkB

# Annotations
!gdown https://drive.google.com/uc?id=1Y6yIcckNonsRZmyD833oCLF-ndu3rvT7 -O Annot_TrainList.txt
!gdown https://drive.google.com/uc?id=13fXL1r62yuVDn6P-QR7GYM4L7d5IdrZa -O Annot_TestList.txt

Retrieving folder list
Processing file 1g0NiXJEibAZdW4W4QAD9_IgCQsZeZ_a5 videos01_txt.tgz
Processing file 1JVuTLp5di33ofDp4njsFhOBPTj8_sZwo videos02_txt.tgz
Processing file 1bR5sBDBlrWCaiVxS8Z7OHACsUVPFo36C videos03_txt.tgz
Processing file 12XQrgHFtRx8Vm3_DIG2vdhjA1OAkSHB0 videos04_txt.tgz
Processing file 1BqQRYWQ2Koz20Zjy9MvDk0K7g--wpSCH videos05_txt.tgz
Retrieving folder list completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1g0NiXJEibAZdW4W4QAD9_IgCQsZeZ_a5
To: /content/videos_txt/videos01_txt.tgz
100% 81.1M/81.1M [00:00<00:00, 108MB/s]
Downloading...
From: https://drive.google.com/uc?id=1JVuTLp5di33ofDp4njsFhOBPTj8_sZwo
To: /content/videos_txt/videos02_txt.tgz
100% 84.0M/84.0M [00:00<00:00, 125MB/s]
Downloading...
From: https://drive.google.com/uc?id=1bR5sBDBlrWCaiVxS8Z7OHACsUVPFo36C
To: /content/videos_txt/videos03_txt.tgz
100% 87.5M/87.5M [00:00<00:00, 109MB/s]
Downloading...
From: https://drive.googl

**Extraction des fichiers compressés**

In [ ]:
import os
import tarfile

def extract_compressed_files_in_folder(compressed_files_folder_path, output_folder_path):
  for file in os.listdir(compressed_files_folder_path):
    with tarfile.open(compressed_files_folder_path + "/" + file) as tar_file:
      tar_file.extractall(output_folder_path)

In [ ]:
extract_compressed_files_in_folder("/content/videos_txt", "/content/videos")

**Constantes**

In [ ]:
LANDMARKS_NUMBER = 21
LANDMARK_DIMENSION = 3

# Ordre des infos des annotations
VIDEO_NAME_PLACE = 0
GESTURE_NAME_PLACE = 1
GESTURE_ID_PLACE = 2
T_START_PLACE = 3
T_END_PLACE = 4
GESTURE_LENGTH_PLACE = 5

**Création des datasets**

In [ ]:
import numpy as np

def remove_newline(strings):
  return [string.replace('\n', '') for string in strings]

def read_txt_file(txt_file_path):
  with open(txt_file_path) as annot_file:
    return remove_newline(annot_file.readlines())

def extract_landmarks(line):
  res = np.zeros((LANDMARKS_NUMBER, LANDMARK_DIMENSION))
  if line != ';':
    split_line = line.split(';')
    for landmark in range(LANDMARKS_NUMBER):
      for dimension in range(LANDMARK_DIMENSION):
        res[landmark][dimension] = float(split_line[landmark*3 + dimension])
  return res

In [ ]:
def get_videos_number(annotations):
  count = 0
  current_video_name = ""
  for info in annotations:
    video_name = info.split(',')[VIDEO_NAME_PLACE]
    if current_video_name != video_name:
      current_video_name = video_name
      count += 1
  return count

def get_maximum_length_gestures(annotations):
  maximum_length = 0
  for info in annotations:
    gesture_length = int(info.split(',')[GESTURE_LENGTH_PLACE])
    if gesture_length > maximum_length:
      maximum_length = gesture_length
  return maximum_length

def get_maximum_length_videos(annotations):
  maximum_length = 0
  for info in annotations:
    t_end = int(info.split(',')[T_END_PLACE])
    if t_end > maximum_length:
      maximum_length = t_end
  return maximum_length

In [ ]:
def create_segmented_dataset(annotations_file_path, print_task_progress=True):
  # Lecture des annotations
  annotations = read_txt_file(annotations_file_path)
  gestures_number = len(annotations)
  maximum_length_gestures = get_maximum_length_gestures(annotations)

  # Initilisation des shapes des résultats
  X = np.zeros((gestures_number, maximum_length_gestures, LANDMARKS_NUMBER, LANDMARK_DIMENSION))
  y = np.zeros(gestures_number)

  # Vidéo actuelle pour éviter de relire le fichier entier à chaque geste de la même vidéo
  current_video_name = ""
  frames = []
  current_video_frames_nb = 0

  for gesture, info in enumerate(annotations):
    # Affichage de l'avancement
    if print_task_progress:
      print("\r", end='')
      print("\rCréation du dataset segmenté - " + "%.2f" % ((gesture + 1) / gestures_number * 100) + " %", end='')

    # Extraction des infos du geste
    video_name = info.split(',')[VIDEO_NAME_PLACE]
    gesture_id = int(info.split(',')[GESTURE_ID_PLACE]) - 1
    t_start = int(info.split(',')[T_START_PLACE]) - 1
    t_end = int(info.split(',')[T_END_PLACE])

    # Changement de la vidéo ?
    if current_video_name != video_name:
      current_video_name = video_name
      current_video_frames = read_txt_file("/content/videos/" + video_name + ".txt")
      current_video_frames_nb = len(current_video_frames)

    # Mise à jour de X avec les coordoonées du geste
    for frame in range(t_start, t_end):
      if frame < current_video_frames_nb:
        landmarks = extract_landmarks(current_video_frames[frame])
        for landmark in range(LANDMARKS_NUMBER):
          for dimension in range(LANDMARK_DIMENSION):
            X[gesture][frame - t_start][landmark][dimension] = landmarks[landmark][dimension]
    
    # Mise à jour de y avec l'id du geste
    y[gesture] = gesture_id
  
  # Affichage des shapes des résultats
  print("\rDataset segmenté - " + annotations_file_path)
  print("Shape X :", X.shape)
  print("Shape y :", y.shape, "\n")
  
  return X, y

In [ ]:
def create_continuous_dataset(annotations_file_path, print_task_progress=True):
  # Lecture des annotations
  annotations = read_txt_file(annotations_file_path)
  gestures_number = len(annotations)
  videos_number = get_videos_number(annotations)
  maximum_length_videos = get_maximum_length_videos(annotations)

  # Initilisation des shapes des résultats
  X = np.zeros((videos_number, maximum_length_videos, LANDMARKS_NUMBER, LANDMARK_DIMENSION))
  y = np.empty((0, 4), dtype=np.int64) # 4 = (numéro de la vidéo, id du geste, début du geste, fin du geste)

  # Vidéo actuelle pour éviter de relire le fichier entier à chaque geste de la même vidéo
  current_video_name = ""
  frames = []
  current_video_frames_nb = 0
  video_id = -1

  for gesture, info in enumerate(annotations):
    # Affichage de l'avancement
    if print_task_progress:
      print("\r", end='')
      print("\rCréation du dataset continu - " + "%.2f" % ((gesture + 1) / gestures_number * 100) + " %", end='')

    # Extraction des infos du geste
    video_name = info.split(',')[VIDEO_NAME_PLACE]
    gesture_id = int(info.split(',')[GESTURE_ID_PLACE]) - 1
    t_start = int(info.split(',')[T_START_PLACE]) - 1
    t_end = int(info.split(',')[T_END_PLACE])

    # Changement de la vidéo
    if current_video_name != video_name:
      current_video_name = video_name
      current_video_frames = read_txt_file("/content/videos/" + video_name + ".txt")
      current_video_frames_nb = len(current_video_frames)
      video_id += 1

    # Mise à jour de X avec les coordoonées du geste
    for frame in range(t_start, t_end):
      if frame < current_video_frames_nb:
        landmarks = extract_landmarks(current_video_frames[frame])
        for landmark in range(LANDMARKS_NUMBER):
          for dimension in range(LANDMARK_DIMENSION):
            X[video_id][frame][landmark][dimension] = landmarks[landmark][dimension]
    
    # Mise à jour de y avec le numéro de la vidéo, l'id du geste, le début du geste et la fin du geste
    infos_y = np.array([video_id, gesture_id, t_start, t_end-1])
    y = np.append(y, infos_y[np.newaxis, :], axis=0)
  
  # Affichage des shapes des résultats
  print("\rDataset continu - " + annotations_file_path)
  print("Shape X :", X.shape)
  print("Shape y :", y.shape, "\n")
  
  return X, y

In [ ]:
X_train_segmented, y_train_segmented = create_segmented_dataset("Annot_TrainList.txt")
X_test_segmented, y_test_segmented = create_segmented_dataset("Annot_TestList.txt")

Dataset segmenté - Annot_TrainList.txt
X shape : (4039, 970, 21, 3)
Y shape : (4039,) 

Dataset segmenté - Annot_TestList.txt
X shape : (1610, 667, 21, 3)
Y shape : (1610,) 



In [ ]:
X_train_continuous, y_train_continuous = create_continuous_dataset("Annot_TrainList.txt")
X_test_continuous, y_test_continuous = create_continuous_dataset("Annot_TestList.txt")

Dataset continu - Annot_TrainList.txt
Shape X : (148, 4905, 21, 3)
Shape y : (4039, 4) 

Dataset continu - Annot_TestList.txt
Shape X : (52, 5038, 21, 3)
Shape y : (1610, 4) 



In [ ]:
print("Vidéo, Geste, Start, End")
print(y_train_continuous[:30])

Vidéo, Geste, Start, End
[[   0    0    0   16]
 [   0   13   17   54]
 [   0    2   55  283]
 [   0    6  284  307]
 [   0    2  308  501]
 [   0    7  502  543]
 [   0    2  544  856]
 [   0    5  857  898]
 [   0    1  899 1121]
 [   0    0 1122 1431]
 [   0    4 1432 1456]
 [   0    1 1457 1706]
 [   0    0 1707 2041]
 [   0   10 2042 2076]
 [   0    1 2077 2349]
 [   0    8 2350 2390]
 [   0    2 2391 2602]
 [   0   12 2603 2645]
 [   0    2 2646 2793]
 [   0    0 2794 2992]
 [   0   11 2993 3028]
 [   0    1 3029 3231]
 [   0    9 3232 3276]
 [   0    1 3277 3644]
 [   0    3 3645 3675]
 [   0    0 3676 3750]
 [   1    0    0   23]
 [   1   12   24   64]
 [   1    2   65  346]
 [   1   13  347  380]]
